Tutorial Followed:

https://snawarhussain.com/blog/computer%20vision/python/tutorial/photometric-stereo-lambertian-model/

In [1]:
import random
import cv2
import glob
import re
import os
import numpy as np